#Init

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import trim,col

# Read from Bronze Table

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")

df.display()

# Data Transformations

In [0]:

# trim the string columns
# normalization of Marital status
# Names are not friendly

df.display()

In [0]:
for field in df.schema.fields:
    if field.dataType == StringType():
        df = df.withColumn(field.name, trim(col(field.name)))
df.display()


## Normalize

In [0]:
df1 =( df.withColumn(
    "cst_marital_status",F.when(F.upper(df.cst_marital_status) == "S", "Single")
    .when(F.upper(df.cst_marital_status) == "M", "Married")
    .otherwise("n/a")
    )

 .withColumn(
    "cst_gndr",F.when(F.upper(df.cst_gndr) == "F", "Female")
    .when(F.upper(df.cst_gndr) == "M", "Male")
    .otherwise("n/a")
    ))





df1.display()


## Renaming of Columns

In [0]:
Rename_Map = {"cst_id":"customer_id",
"cst_key":"customer_key",
"cst_firstname":"first_name",
"cst_lastname":'last_name',
'cst_marital_status':'marital_status',
'cst_gndr':'gender',
'cst_create_date':'create_date'}

In [0]:
for col in Rename_Map:
    df1 = df1.withColumnRenamed(col,Rename_Map[col])

df1.display()

# Write into Silver Table

In [0]:
(df1.write.mode("overwrite")
.format("delta")
.saveAsTable("workspace.silver.crm_customer"))